In [1]:
import pandas as pd

In [2]:
#Robert's code for importing csv
file = 'Resources/MN_Accidents_Dec20.csv'
df = pd.read_csv(file)
df.head()


,Unnamed: 0,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,512821,A-512837,MapQuest,201.0,2,2020-12-31 04:35:10,2020-12-31 05:07:00,44.805157,-92.913040,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,512822,A-512838,MapQuest,201.0,2,2020-12-31 05:07:21,2020-12-31 05:39:00,44.658508,-92.771248,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
2,512824,A-512840,MapQuest,201.0,2,2020-12-31 07:42:22,2020-12-31 08:39:49,44.775558,-92.863136,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
3,512825,A-512841,MapQuest,201.0,2,2020-12-31 08:09:43,2020-12-31 08:59:00,44.952919,-93.088249,NaN,...,False,False,False,False,False,False,Day,Day,Day,Day
4,512827,A-512843,MapQuest,201.0,2,2020-12-31 12:31:49,2020-12-31 13:34:30,44.954311,-93.400703,NaN,...,False,False,False,False,False,False,Day,Day,Day,Day


In [14]:
#delete all unneccesary columns
df = df.drop(["Source","End_Lat","End_Lng","Distance(mi)","Side","Zipcode","Country","Timezone","Airport_Code","Weather_Timestamp","Wind_Chill(F)","Humidity(%)","Pressure(in)","Wind_Direction","Wind_Speed(mph)","Precipitation(in)","Amenity","Bump","Crossing","Give_Way","Junction","No_Exit","Railway","Roundabout","Station","Stop","Traffic_Calming","Turning_Loop","Sunrise_Sunset","Civil_Twilight","Nautical_Twilight","Astronomical_Twilight"], axis=1)


In [15]:
df.head()

,Unnamed: 0,ID,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Description,Number,Street,City,County,State,Temperature(F),Visibility(mi),Weather_Condition,Traffic_Signal
0,512821,A-512837,201.0,2,2020-12-31 04:35:10,2020-12-31 05:07:00,44.805157,-92.913040,Accident on US-61 Point Douglas Rd Northbound ...,NaN,US-10 E,Cottage Grove,Washington,MN,11.0,10.0,Mostly Cloudy,False
1,512822,A-512838,201.0,2,2020-12-31 05:07:21,2020-12-31 05:39:00,44.658508,-92.771248,Accident on MN-316 Red Wing Blvd at CR-18 200t...,16700.0,200th St E,Hastings,Dakota,MN,13.0,10.0,Cloudy,False
2,512824,A-512840,201.0,2,2020-12-31 07:42:22,2020-12-31 08:39:49,44.775558,-92.863136,Accident on US-10 Point Douglas Dr Northbound ...,11927.0,Point Douglas Dr S,Hastings,Washington,MN,17.0,10.0,Cloudy,False
3,512825,A-512841,201.0,2,2020-12-31 08:09:43,2020-12-31 08:59:00,44.952919,-93.088249,Entry ramp to US-52 Southbound from I-35E Sout...,NaN,US-10 E,Saint Paul,Ramsey,MN,17.0,10.0,Cloudy,False
4,512827,A-512843,201.0,2,2020-12-31 12:31:49,2020-12-31 13:34:30,44.954311,-93.400703,Right lane blocked and right hand shoulder blo...,NaN,US-169 N,Hopkins,Hennepin,MN,23.0,10.0,Cloudy,False


In [8]:
#df.head()
#change start time object into datetime format for filtering
df['Start_Time'] = pd.to_datetime(df['Start_Time'])
df.dtypes

Unnamed: 0                        int64
ID                               object
Source                           object
TMC                             float64
Severity                          int64
Start_Time               datetime64[ns]
End_Time                         object
Start_Lat                       float64
Start_Lng                       float64
End_Lat                         float64
End_Lng                         float64
Distance(mi)                    float64
Description                      object
Number                          float64
Street                           object
Side                             object
City                             object
County                           object
State                            object
Zipcode                          object
Country                          object
Timezone                         object
Airport_Code                     object
Weather_Timestamp                object
Temperature(F)                  float64
